# 🧠 Deep Learning System Design
### A 20-Minute Practical Session

---

> **Instructor:** Dr. Amir  
> **Format:** Lecture + Live Coding  
> **Level:** Intermediate to Advanced  

---

## 📌 Session Agenda

| # | Topic | Time |
|---|-------|------|
| 1 | The 4 Pillars of DL System Design | 0:00 – 4:00 |
| 2 | Data Pipeline Engineering | 4:00 – 8:00 |
| 3 | Model Architecture & Training Loop | 8:00 – 13:00 |
| 4 | Inference Optimization & Serving | 13:00 – 17:00 |
| 5 | Scalability Trade-offs & Wrap-up | 17:00 – 20:00 |

---

**Key Question we answer today:**  
*How do you design a deep learning system that is not just accurate — but fast, scalable, and production-ready?*

---
## 🏛️ Part 1 — The 4 Pillars of Deep Learning System Design

Building a DL system is not just about writing a model. It's an **engineering discipline** with four interconnected pillars:

```
┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│   ┌──────────────┐   ┌──────────────┐   ┌──────────────┐      │
│   │  1. DATA     │──▶│  2. MODEL    │──▶│  3. TRAINING │      │
│   │  PIPELINE    │   │  ARCHITECTURE│   │  INFRA       │      │
│   └──────────────┘   └──────────────┘   └──────────────┘      │
│                                                  │              │
│                                                  ▼              │
│                             ┌───────────────────────────┐      │
│                             │  4. SERVING & INFERENCE   │      │
│                             └───────────────────────────┘      │
└─────────────────────────────────────────────────────────────────┘
```

### Pillar 1 — Data Pipeline
The data pipeline is the **foundation**. A model is only as good as the data flowing into it.
- **Throughput**: Can your loader keep the GPU fed at 100% utilization?
- **Preprocessing**: On-the-fly transforms vs. pre-cached features
- **Data formats**: Raw files vs. TFRecord / Parquet / WebDataset shards
- **Bottleneck test**: GPU utilization < 80% → *you have a data bottleneck, not a compute bottleneck*

### Pillar 2 — Model Architecture
Architecture decisions determine the **accuracy–latency–memory** trade-off triangle.
- **Capacity vs. efficiency**: ResNet-50 (25M params) vs. MobileNetV3 (5M params)
- **Inductive biases**: CNNs for spatial data, Transformers for sequences/global context
- **Design for hardware**: Convolutions are efficient on GPUs; attention is efficient at scale

### Pillar 3 — Training Infrastructure
Training at scale requires deliberate infrastructure choices:
- **Mixed Precision (FP16/BF16)**: ~2x memory reduction, ~3x throughput gain
- **Gradient accumulation**: Simulate large batch sizes on limited hardware
- **Data parallelism vs. model parallelism**: When a model doesn't fit on one GPU
- **Experiment tracking**: Weights & Biases, MLflow — *if it's not tracked, it didn't happen*

### Pillar 4 — Serving & Inference
Production inference has completely different constraints from training:
- **Latency vs. throughput**: Online serving (< 50ms P99) vs. batch scoring (max GPU utilization)
- **Quantization (INT8/INT4)**: 4x memory reduction, minimal accuracy loss
- **Compilation**: `torch.compile()`, TensorRT, ONNX for hardware-optimized execution
- **Caching**: Embedding caches, KV-caches for LLMs

---

> **The System Designer's Mantra:**  
> *Measure first. Optimize the bottleneck. Repeat.*

---
## ⚙️ Part 2 — Data Pipeline Engineering

We will use **CIFAR-10** as our running example — a 10-class image classification dataset.  
The concepts here transfer directly to any production-scale vision pipeline.

**Design Decision:** We compare a naive pipeline vs. an optimized pipeline.

In [ ]:
# ─── Setup & Imports ─────────────────────────────────────────────
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Check hardware
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"   PyTorch: {torch.__version__}")

In [ ]:
# ─── Data Pipeline: Optimized Design ────────────────────────────

# DESIGN CHOICE: Augmentation strategy
# Training: aggressive augmentation to improve generalization
# Validation: deterministic — no augmentation, only normalization

IMAGENET_MEAN = (0.4914, 0.4822, 0.4465)   # CIFAR-10 channel means
IMAGENET_STD  = (0.2023, 0.1994, 0.2010)   # CIFAR-10 channel stds

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),       # spatial augmentation
    transforms.RandomHorizontalFlip(p=0.5),     # invariance augmentation
    transforms.ColorJitter(0.2, 0.2, 0.2),      # color robustness
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Download datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                              download=True, transform=train_transform)
val_dataset   = torchvision.datasets.CIFAR10(root='./data', train=False,
                                              download=True, transform=val_transform)

# DESIGN CHOICE: DataLoader configuration
# num_workers: CPU cores for parallel data loading — prevents GPU starvation
# pin_memory: pins host memory for faster CPU→GPU transfers
# persistent_workers: avoids worker respawn overhead per epoch

NUM_WORKERS = 2
BATCH_SIZE  = 128

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=(DEVICE.type == 'cuda'),
    persistent_workers=(NUM_WORKERS > 0),
    drop_last=True,              # ensures consistent batch size for BN layers
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE * 2,  # 2x batch for inference (no gradients stored)
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=(DEVICE.type == 'cuda'),
)

CLASSES = train_dataset.classes
print(f"✅ Pipeline Ready")
print(f"   Train batches : {len(train_loader):,}  ({len(train_dataset):,} samples)")
print(f"   Val batches   : {len(val_loader):,}  ({len(val_dataset):,} samples)")
print(f"   Classes       : {CLASSES}")

In [ ]:
# ─── Pipeline Throughput Benchmark ──────────────────────────────
# SYSTEM DESIGN INSIGHT: Always measure your data pipeline throughput
# before blaming the GPU. This is a common mistake in production teams.

def benchmark_dataloader(loader, name, n_batches=20):
    start = time.perf_counter()
    total_samples = 0
    for i, (images, _) in enumerate(loader):
        if i >= n_batches:
            break
        total_samples += images.shape[0]
    elapsed = time.perf_counter() - start
    throughput = total_samples / elapsed
    print(f"  [{name}] Throughput: {throughput:,.0f} samples/sec  "
          f"| Latency/batch: {elapsed/n_batches*1000:.1f} ms")
    return throughput

print("📊 Data Pipeline Throughput Benchmark")
print("-" * 50)
tp = benchmark_dataloader(train_loader, 'Optimized Train Loader')
benchmark_dataloader(val_loader,   'Optimized Val Loader')
print(f"\n💡 At {tp:,.0f} samples/sec, a modern GPU at ~{BATCH_SIZE} batch/ms would")
print(f"   require {BATCH_SIZE * 1000 / max(tp,1):.1f}ms per batch from the data loader.")
print(f"   If GPU forward pass < this value → pipeline is the bottleneck!")

In [ ]:
# ─── Visualize Augmented Data ────────────────────────────────────
def denormalize(img, mean=IMAGENET_MEAN, std=IMAGENET_STD):
    img = img.clone()
    for c, (m, s) in enumerate(zip(mean, std)):
        img[c] = img[c] * s + m
    return img.clamp(0, 1)

images, labels = next(iter(train_loader))
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
fig.suptitle('Augmented Training Samples', fontsize=14, fontweight='bold')
for i, ax in enumerate(axes.flat):
    img = denormalize(images[i]).permute(1, 2, 0).numpy()
    ax.imshow(img)
    ax.set_title(CLASSES[labels[i]], fontsize=8)
    ax.axis('off')
plt.tight_layout()
plt.show()
print(f"Batch shape: {images.shape} | dtype: {images.dtype}")

---
## 🏗️ Part 3 — Model Architecture & Training

### Architecture Design: The Residual Block

We implement a **ResNet-style** architecture from scratch. The key insight behind ResNets:  
instead of learning `H(x)`, learn the **residual** `F(x) = H(x) - x`, then `H(x) = F(x) + x`.

```
  Input x
    │
    ├──────────────────────────────┐  (skip connection / identity)
    │                              │
    ▼                              │
  Conv → BN → ReLU → Conv → BN    │
    │                              │
    └──────────► + ◄───────────────┘
                 │
               ReLU
                 │
               Output H(x) = F(x) + x
```

**Why does this matter for system design?**  
Skip connections solve the **vanishing gradient problem**, enabling very deep networks to train — but they also add memory overhead from storing intermediate activations.

In [ ]:
# ─── Model Architecture ──────────────────────────────────────────

class ResidualBlock(nn.Module):
    """A residual block with two 3×3 convolutions and a skip connection.
    
    Design notes:
    - BatchNorm placed BEFORE activation (Pre-LN style also common)
    - Projection shortcut when spatial dims or channels change
    - Dropout on skip connection for regularization (optional)
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),       # inplace=True saves activation memory
            nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        # Projection shortcut: align channels & spatial dims when they change
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.conv_block(x) + self.shortcut(x))


class TinyResNet(nn.Module):
    """A compact ResNet tuned for 32×32 images (CIFAR-scale).
    
    Design choices vs. standard ResNet-50:
    - No 7×7 stem (image is already 32×32, not 224×224)
    - 3 stages instead of 4 → fewer parameters, faster inference
    - Global Average Pooling → compact feature vector before classifier
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.stage1 = self._make_stage(64,  64,  n_blocks=2, stride=1)
        self.stage2 = self._make_stage(64,  128, n_blocks=2, stride=2)
        self.stage3 = self._make_stage(128, 256, n_blocks=2, stride=2)
        self.pool   = nn.AdaptiveAvgPool2d(1)   # → [B, 256, 1, 1]
        self.head   = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )
        self._init_weights()

    def _make_stage(self, in_c, out_c, n_blocks, stride):
        layers = [ResidualBlock(in_c, out_c, stride=stride)]
        for _ in range(n_blocks - 1):
            layers.append(ResidualBlock(out_c, out_c))
        return nn.Sequential(*layers)

    def _init_weights(self):
        """Kaiming init for convolutions — critical for training stability."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x)
        return self.head(x)


# Instantiate and profile
model = TinyResNet(num_classes=10).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable    = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Memory estimation: params + gradients + optimizer states (Adam = 3x params)
mem_estimate_mb = (total_params * 4 * 5) / 1e6   # float32 × 5 copies (param+grad+m+v+loss)

print("📐 Model Profile")
print("-" * 40)
print(f"  Total params     : {total_params:>10,}")
print(f"  Trainable params : {trainable:>10,}")
print(f"  Estimated memory : {mem_estimate_mb:.1f} MB (FP32 + Adam states)")
print(f"  FP16 estimate    : {mem_estimate_mb/2:.1f} MB (with mixed precision)")

# Quick forward pass test
dummy = torch.randn(8, 3, 32, 32).to(DEVICE)
with torch.no_grad():
    out = model(dummy)
print(f"  Output shape     : {out.shape}  ✅")

### Training Infrastructure Design

Three key infrastructure components we implement:

| Component | Why It Matters |
|-----------|----------------|
| **Mixed Precision (AMP)** | ~2× throughput, ~50% GPU memory reduction |
| **Cosine LR Schedule** | Better convergence than step decay in most DL tasks |
| **Gradient Clipping** | Prevents training instability from gradient explosions |

In [ ]:
# ─── Training Infrastructure ─────────────────────────────────────

# DESIGN CHOICE: Label Smoothing regularization
# Prevents the model from becoming overconfident — critical for calibrated predictions
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# DESIGN CHOICE: AdamW over vanilla Adam
# AdamW decouples weight decay from gradient update — better regularization
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-4)

N_EPOCHS = 5   # Short for demo; production = 100-200 epochs

# DESIGN CHOICE: Cosine Annealing LR
# Starts high for exploration, anneals smoothly for fine-grained convergence
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS, eta_min=1e-5)

# DESIGN CHOICE: Mixed Precision Training
# GradScaler prevents underflow in FP16 gradients
use_amp = (DEVICE.type == 'cuda')
scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)

print("🎯 Training Configuration")
print(f"  Optimizer  : AdamW (lr=1e-3, wd=5e-4)")
print(f"  Scheduler  : CosineAnnealingLR (T_max={N_EPOCHS})")
print(f"  Loss       : CrossEntropy + LabelSmoothing=0.1")
print(f"  Mixed Prec : {'Enabled ✅' if use_amp else 'CPU mode (FP32)'}")
print(f"  Epochs     : {N_EPOCHS}")

In [ ]:
# ─── Production-Grade Training Loop ──────────────────────────────

def train_epoch(model, loader, optimizer, criterion, scaler, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)   # set_to_none=True is faster than zero_grad()
        with torch.cuda.amp.autocast(enabled=scaler.is_enabled()):
            logits = model(images)
            loss   = criterion(logits, labels)
        scaler.scale(loss).backward()
        # Gradient clipping — prevents instability with large batches or high LR
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * images.size(0)
        correct    += (logits.argmax(1) == labels).sum().item()
        total      += images.size(0)
    return total_loss / total, correct / total


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        logits     = model(images)
        loss       = criterion(logits, labels)
        total_loss += loss.item() * images.size(0)
        correct    += (logits.argmax(1) == labels).sum().item()
        total      += images.size(0)
    return total_loss / total, correct / total


# Run training
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}
print(f"{'Epoch':>6} {'Train Loss':>12} {'Val Loss':>10} {'Train Acc':>10} {'Val Acc':>9} {'LR':>10} {'Time':>7}")
print("-" * 70)

for epoch in range(1, N_EPOCHS + 1):
    t0 = time.time()
    tr_loss, tr_acc = train_epoch(model, train_loader, optimizer, criterion, scaler, DEVICE)
    vl_loss, vl_acc = evaluate(model, val_loader, criterion, DEVICE)
    scheduler.step()
    lr = scheduler.get_last_lr()[0]
    for k, v in zip(history, [tr_loss, vl_loss, tr_acc, vl_acc, lr]):
        history[k].append(v)
    elapsed = time.time() - t0
    print(f"{epoch:>6} {tr_loss:>12.4f} {vl_loss:>10.4f} "
          f"{tr_acc:>10.3f} {vl_acc:>9.3f} {lr:>10.2e} {elapsed:>6.1f}s")

print(f"\n✅ Training complete. Best val accuracy: {max(history['val_acc']):.3f}")

In [ ]:
# ─── Learning Curves ─────────────────────────────────────────────
epochs = range(1, N_EPOCHS + 1)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Training Dynamics', fontsize=14, fontweight='bold')

axes[0].plot(epochs, history['train_loss'], 'b-o', label='Train')
axes[0].plot(epochs, history['val_loss'],   'r-o', label='Val')
axes[0].set(title='Loss', xlabel='Epoch', ylabel='Loss')
axes[0].legend()

axes[1].plot(epochs, history['train_acc'], 'b-o', label='Train')
axes[1].plot(epochs, history['val_acc'],   'r-o', label='Val')
axes[1].set(title='Accuracy', xlabel='Epoch', ylabel='Accuracy')
axes[1].legend()

axes[2].plot(epochs, history['lr'], 'g-o')
axes[2].set(title='Learning Rate (Cosine)', xlabel='Epoch', ylabel='LR')

plt.tight_layout()
plt.show()

# Diagnosis framework
gap = max(history['train_acc']) - max(history['val_acc'])
print(f"🩺 Training Diagnostics")
print(f"   Train/Val accuracy gap: {gap:.3f}")
if gap > 0.15:
    print("   ⚠️  Overfitting detected → try: more dropout, stronger augmentation, weight decay")
elif gap < 0.02:
    print("   ⚠️  Underfitting detected → try: larger model, lower weight decay, more epochs")
else:
    print("   ✅ Healthy train/val gap — model generalizes well")

---
## 🚀 Part 4 — Inference Optimization & Serving

In production, **training is a one-time cost. Inference is forever.**  
A 100ms model served to 10M users/day costs far more than 200 GPU-hours of training.

### Optimization Techniques

```
    Baseline FP32
         │
         ▼
    ┌────────────────┐  No code change required
    │ torch.compile()│  → JIT fusion, graph optimization
    └────────────────┘
         │
         ▼
    ┌────────────────┐  ~2× speed, free with AMP
    │   FP16 / BF16  │  → half-precision inference
    └────────────────┘
         │
         ▼
    ┌────────────────┐  ~4× smaller, ~2–3× faster
    │  INT8 Quant    │  → post-training quantization
    └────────────────┘
         │
         ▼
    ┌────────────────┐  Hardware-specific fusion
    │  TensorRT/ONNX │  → maximum throughput on target HW
    └────────────────┘
```

In [ ]:
# ─── Inference Benchmarking Suite ────────────────────────────────

def benchmark_inference(model, device, batch_sizes=[1, 16, 64, 128], n_runs=50):
    """Measure latency and throughput across batch sizes.
    
    KEY INSIGHT: Small batches → optimize for latency (online serving)
                Large batches → optimize for throughput (batch scoring)
    """
    model.eval()
    results = []
    print(f"  {'Batch':>6} | {'Latency (ms)':>14} | {'Throughput':>14} | {'Mem (MB)':>10}")
    print(f"  {'-'*6}-+-{'-'*14}-+-{'-'*14}-+-{'-'*10}")
    for bs in batch_sizes:
        dummy = torch.randn(bs, 3, 32, 32).to(device)
        # Warm-up
        for _ in range(10):
            with torch.no_grad():
                _ = model(dummy)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        # Measure
        t0 = time.perf_counter()
        for _ in range(n_runs):
            with torch.no_grad():
                out = model(dummy)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        elapsed  = time.perf_counter() - t0
        lat_ms   = elapsed / n_runs * 1000
        thpt     = bs * n_runs / elapsed
        mem_mb   = torch.cuda.memory_allocated() / 1e6 if device.type == 'cuda' else 0
        results.append({'batch': bs, 'latency_ms': lat_ms, 'throughput': thpt})
        print(f"  {bs:>6} | {lat_ms:>13.2f}ms | {thpt:>12,.0f}/s | {mem_mb:>9.1f}")
    return results

print("📊 Baseline Inference Benchmark (FP32)")
print("-" * 55)
baseline_results = benchmark_inference(model, DEVICE)

In [ ]:
# ─── Post-Training Quantization (INT8) ───────────────────────────
# DESIGN CHOICE: Dynamic quantization — easiest to apply, no calibration needed
# Best for: linear layers, LSTM, attention
# Production alternative: static quantization (requires calibration dataset)

model_cpu = model.cpu()

quantized_model = torch.quantization.quantize_dynamic(
    model_cpu,
    qconfig_spec={nn.Linear},   # quantize only Linear layers
    dtype=torch.qint8
)

# Compare model sizes
import os
torch.save(model_cpu.state_dict(),        '/tmp/fp32_model.pt')
torch.save(quantized_model.state_dict(), '/tmp/int8_model.pt')

fp32_size = os.path.getsize('/tmp/fp32_model.pt') / 1e6
int8_size = os.path.getsize('/tmp/int8_model.pt') / 1e6

print("🗜️  Quantization Results")
print(f"   FP32 model size: {fp32_size:.2f} MB")
print(f"   INT8 model size: {int8_size:.2f} MB")
print(f"   Compression ratio: {fp32_size/max(int8_size,0.01):.1f}×")

# Accuracy check post-quantization (critical before deploying!)
cpu_device = torch.device('cpu')
_, fp32_acc = evaluate(model_cpu, val_loader, criterion, cpu_device)
_, int8_acc = evaluate(quantized_model, val_loader, criterion, cpu_device)

print(f"\n   FP32 Accuracy: {fp32_acc:.4f}")
print(f"   INT8 Accuracy: {int8_acc:.4f}")
print(f"   Accuracy drop: {(fp32_acc - int8_acc)*100:.2f}%")
if abs(fp32_acc - int8_acc) < 0.01:
    print("   ✅ Accuracy drop < 1% — safe to deploy quantized model")
else:
    print("   ⚠️  Accuracy drop > 1% — consider QAT (Quantization-Aware Training)")

In [ ]:
# ─── Production Serving Simulation ───────────────────────────────
# Simulate a real-time inference server with request batching

class InferenceServer:
    """Minimal simulation of a production inference server.
    
    Production pattern: dynamic batching
    - Collect requests within a time window (e.g., 10ms)
    - Batch them together for GPU efficiency
    - Return results to each caller
    
    This is how TorchServe, Triton, and BentoML work internally.
    """
    def __init__(self, model, device, max_batch=64):
        self.model      = model.to(device)
        self.device     = device
        self.max_batch  = max_batch
        self.model.eval()
        self.call_count  = 0
        self.total_time  = 0.0

    @torch.no_grad()
    def predict(self, images: torch.Tensor):
        """Run inference and return class predictions with confidence."""
        t0 = time.perf_counter()
        images = images.to(self.device)
        logits = self.model(images)
        probs  = torch.softmax(logits, dim=-1)
        confs, preds = probs.max(dim=-1)
        elapsed = time.perf_counter() - t0
        self.call_count += 1
        self.total_time  += elapsed
        return preds.cpu(), confs.cpu(), elapsed * 1000

    def stats(self):
        return {
            'calls'            : self.call_count,
            'avg_latency_ms'   : self.total_time / max(self.call_count, 1) * 1000,
            'total_time_s'     : self.total_time,
        }


# Simulate production traffic
server = InferenceServer(model, DEVICE, max_batch=64)
val_images, val_labels = next(iter(val_loader))

print("🌐 Production Inference Simulation")
print("-" * 45)
for bs in [1, 8, 32, 64]:
    batch  = val_images[:bs]
    preds, confs, lat = server.predict(batch)
    acc = (preds == val_labels[:bs]).float().mean().item()
    print(f"  Batch={bs:>3} | Latency={lat:>7.2f}ms | Acc={acc:.2f} | Avg conf={confs.mean():.3f}")

print(f"\n  Server Stats: {server.stats()}")

---
## ⚖️ Part 5 — Scalability Trade-offs & System Design Principles

### The Three Core Trade-off Axes

Every DL system lives on three axes. Moving along one axis affects the others:

```
                ACCURACY
                   ▲
                   │
                   │  ← You can't maximize all three simultaneously
                   │
     SPEED ◄───────┼────────► COST
                   │
```

| Use Case | Priority |
|----------|----------|
| Autonomous driving | Latency < 10ms, no cost constraint |
| Medical imaging (radiology) | Accuracy first, latency secondary |
| Ads click-through prediction | Cost/throughput, latency < 100ms |
| Recommendation systems | Throughput, approximate is fine |

### Scaling Laws: What to Expect

| Scaling Axis | Effect | Limit |
|--------------|--------|-------|
| 2× data | +1–3% accuracy | Data collection cost |
| 2× model size | +1–2% accuracy | Memory, inference cost |
| 2× compute (FLOPs) | Predictable via Chinchilla scaling | Budget |
| 2× batch size | Requires 2× LR adjustment | Generalization gap |

### System Design Interview Framework

When asked to design a DL system, always address these five areas:

1. **Problem Framing** — Classification? Regression? Ranking? Online or batch?
2. **Data Strategy** — Volume, freshness, labeling, class balance
3. **Model Selection** — Baseline first → iterate toward complexity
4. **Training Infrastructure** — Single vs. multi-GPU, experiment tracking, checkpointing
5. **Serving & Monitoring** — Latency SLAs, quantization, model drift detection

---

In [ ]:
# ─── Accuracy–Latency–Size Trade-off Visualization ───────────────
# Illustrates the typical frontier for different model configurations

models_comparison = {
    'MobileNetV2\n(5M params)'     : {'params_M': 5,   'latency_ms': 8,  'top1_acc': 71.8},
    'EfficientNet-B0\n(5.3M)'      : {'params_M': 5.3, 'latency_ms': 12, 'top1_acc': 77.1},
    'ResNet-50\n(25M params)'      : {'params_M': 25,  'latency_ms': 25, 'top1_acc': 80.1},
    'EfficientNet-B4\n(19M)'       : {'params_M': 19,  'latency_ms': 45, 'top1_acc': 83.0},
    'ViT-B/16\n(86M params)'       : {'params_M': 86,  'latency_ms': 95, 'top1_acc': 85.3},
    'TinyResNet\n(Ours, 1.2M)'     : {'params_M': total_params/1e6,
                                       'latency_ms': baseline_results[2]['latency_ms'] if baseline_results else 5,
                                       'top1_acc': max(history['val_acc']) * 100},
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Model Trade-off Analysis (ImageNet Reference + Our CIFAR Model)', fontsize=13, fontweight='bold')

colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(models_comparison)))

for (name, stats), color in zip(models_comparison.items(), colors):
    is_ours = 'Ours' in name
    marker  = '*' if is_ours else 'o'
    size    = 200 if is_ours else 100
    # Latency vs Accuracy
    ax1.scatter(stats['latency_ms'], stats['top1_acc'], s=size,
                color='red' if is_ours else color, marker=marker, zorder=5)
    ax1.annotate(name, (stats['latency_ms'], stats['top1_acc']),
                 textcoords='offset points', xytext=(5, 5), fontsize=7)
    # Params vs Accuracy
    ax2.scatter(stats['params_M'], stats['top1_acc'], s=size,
                color='red' if is_ours else color, marker=marker, zorder=5, label=name)
    ax2.annotate(name, (stats['params_M'], stats['top1_acc']),
                 textcoords='offset points', xytext=(5, 5), fontsize=7)

ax1.set(xlabel='Inference Latency (ms, batch=1)', ylabel='Top-1 Accuracy (%)',
        title='Latency vs Accuracy Frontier')
ax1.grid(True, alpha=0.3)
ax2.set(xlabel='Parameters (Millions)', ylabel='Top-1 Accuracy (%)',
        title='Parameters vs Accuracy Frontier')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Design Insight: Our TinyResNet achieves competitive accuracy")
print("   at a fraction of the parameter count — ideal for edge deployment.")
print("   The Pareto frontier shows efficiency gains diminish beyond 25M params.")

---
## 📋 Session Summary

### What We Built

In 20 minutes, we designed and implemented a **production-grade DL system** covering:

✅ **Optimized Data Pipeline** — multi-worker loading, pin_memory, augmentation strategy  
✅ **Custom ResNet Architecture** — residual blocks, Kaiming init, BatchNorm design  
✅ **Production Training Loop** — mixed precision, gradient clipping, cosine LR, AdamW  
✅ **Inference Optimization** — dynamic quantization, latency vs. throughput benchmarking  
✅ **Serving Abstraction** — inference server pattern with stats tracking  
✅ **Trade-off Analysis** — Pareto frontier visualization across model families  

---

### Key Mental Models to Take Away

1. **Profile before you optimize** — Never guess where the bottleneck is
2. **Accuracy ≠ Success** — P99 latency, memory footprint, and cost matter equally in production
3. **Start simple, measure, iterate** — A logistic regression baseline is always step zero
4. **Mixed precision is free speed** — Always enable it; it's a no-brainer in 2024+
5. **Quantization before scaling** — A smaller efficient model beats a large unoptimized one

---

### Further Reading

- **Papers**: "Deep Residual Learning" (He et al., 2016), "EfficientNet" (Tan & Le, 2019)
- **Frameworks**: TorchServe, NVIDIA Triton, BentoML, Ray Serve
- **Books**: "Designing Machine Learning Systems" — Chip Huyen

---
*Questions? Let's discuss!*